In [10]:
# Apurva Shah, 705595011, Psych 186B
# Homework 5

# General Imports
import random
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import matplotlib as mpl
import math
import statistics

In [11]:
def widrow_hoff(correct_g, actual_g, f, k):
    # Calculates the Widrow-Hoff delta for updating the weight matrix in learning algorithms.
    # This function implements the Widrow-Hoff learning rule, also known as the Least Mean Squares (LMS) rule,
    # which is used for error correction in adaptive filters and neural networks. It adjusts the weights
    # based on the difference between the correct output and the actual output, scaled by a learning rate.
    
    difference_vector = correct_g - actual_g
    weighted_vector = k * difference_vector
    delta_A = np.outer(weighted_vector, f.T)
    return delta_A

In [12]:
def plot_weight_matrix_changes(A_history):
    """
    Visualizes the change in the weight matrix over iterations.
    
    Args:
    - A_history: A list of A matrices over iterations.
    """
    fig, ax = plt.subplots(figsize=(10, 6))
    changes = [np.linalg.norm(A_history[i] - A_history[i-1]) for i in range(1, len(A_history))]
    ax.plot(changes)
    ax.set_title('Change in Weight Matrix Over Iterations')
    ax.set_xlabel('Iteration')
    ax.set_ylabel('Change in Weight Matrix Norm')
    plt.show()

def plot_feature_distributions(ships, num_ships, feature_names):
    """
    Plots histograms for each feature across different ship origins.
    
    Args:
    - ships: Numpy array of ships data.
    - num_ships: Number of ships.
    - feature_names: List of feature names.
    """
    origins = ['Klingon', 'Romulan', 'Antarean', 'Federation']
    fig, axes = plt.subplots(len(feature_names), 1, figsize=(10, 15))
    for i, feature_name in enumerate(feature_names):
        for origin_index, origin in enumerate(origins, start=1):
            data = ships[:num_ships, i][ships[6, :num_ships] == origin_index]
            axes[i].hist(data, label=origin, alpha=0.5)
        axes[i].set_title(f'Distribution of {feature_name}')
        axes[i].legend()
    plt.tight_layout()
    plt.show()

def plot_confusion_matrix(predictions, true_labels):
    """
    Visualizes the confusion matrix.
    
    Args:
    - predictions: Numpy array of predicted labels.
    - true_labels: Numpy array of true labels.
    """
    from sklearn.metrics import confusion_matrix
    import seaborn as sns
    cm = confusion_matrix(true_labels, predictions)
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
    plt.title('Confusion Matrix')
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.show()

In [13]:
# Import & Clean Data
ships = pd.read_csv('traindataConverted.csv', header=None).to_numpy()
noisy_ships = pd.read_csv('testdataConverted.csv', header=None).to_numpy()

ships = ships.astype(float)
noisy_ships = noisy_ships.astype(float)

num_features = 6  # number of features
num_ships = 20

In [14]:
# Normalize feature vectors
for i in range(num_ships):
    ships[:6, i] /= np.linalg.norm(ships[:6, i])
    noisy_ships[:6, i] /= np.linalg.norm(noisy_ships[:6, i])

# Calculate mean vectors for each origin
klingons = ships[:, :5]
k_mean = np.mean(klingons, axis=1)

romulans = ships[:, 5:10]
r_mean = np.mean(romulans, axis=1)

antareans = ships[:, 10:15]
a_mean = np.mean(antareans, axis=1)

federations = ships[:, 15:]
f_mean = np.mean(federations, axis=1)

In [15]:
# Learning Parameter Setup
k = 0.1
A = np.zeros((6, 6))
np.random.seed()
num_iterations = 10000

In [16]:
# Conduct Learning for Widrow Hoff
A_history = []
for i in range(num_iterations):
    index = np.random.randint(20)
    f = ships[:6, index]
    f /= np.linalg.norm(f)

    
    origin = ships[6, index]
    if origin == 1:
        g = k_mean[:6]
    elif origin == 2:
        g = r_mean[:6]
    elif origin == 3:
        g = a_mean[:6]
    elif origin == 4:
        g = f_mean[:6]
    
    Ai = np.outer(g, f)
    A += Ai
    g_i = np.dot(A, f)
    g_i /= np.linalg.norm(g_i)
    d_A = widrow_hoff(g, g_i, f, k)
    A += d_A
    A_history.append(A.copy())

In [17]:
# Apply Model for Fuzzy/Incomplete Data
predictions = np.zeros(num_ships)

for j in range(num_ships):
    f = noisy_ships[:6, j]
    g_i = np.dot(A, f)
    g_i /= np.linalg.norm(g_i)
    
    dif_k = abs(1 - np.dot(g_i, k_mean[:6]))
    dif_r = abs(1 - np.dot(g_i, r_mean[:6]))
    dif_a = abs(1 - np.dot(g_i, a_mean[:6]))
    dif_f = abs(1 - np.dot(g_i, f_mean[:6]))
    
    differences = [dif_k, dif_r, dif_a, dif_f]
    prediction = np.argmin(differences) + 1
    
    predictions[j] = prediction

In [18]:
# Print Outputs for Predictions

for k in range(num_ships):
    prediction = predictions[k]
    if prediction == 1:
        origin = 'Klingon'
        action = 'Hostile'
    elif prediction == 2:
        origin = 'Romulan'
        action = 'Alert'
    elif prediction == 3:
        origin = 'Antarean'
        action = 'Friendly'
    elif prediction == 4:
        origin = 'Federation'
        action = 'Friendly'
    
    print(f'Ship origin: {origin}\t\t Required action: {action}')

Ship origin: Klingon		 Required action: Hostile
Ship origin: Federation		 Required action: Friendly
Ship origin: Federation		 Required action: Friendly
Ship origin: Antarean		 Required action: Friendly
Ship origin: Klingon		 Required action: Hostile
Ship origin: Klingon		 Required action: Hostile
Ship origin: Klingon		 Required action: Hostile
Ship origin: Romulan		 Required action: Alert
Ship origin: Klingon		 Required action: Hostile
Ship origin: Antarean		 Required action: Friendly
Ship origin: Klingon		 Required action: Hostile
Ship origin: Federation		 Required action: Friendly
Ship origin: Antarean		 Required action: Friendly
Ship origin: Antarean		 Required action: Friendly
Ship origin: Romulan		 Required action: Alert
Ship origin: Romulan		 Required action: Alert
Ship origin: Antarean		 Required action: Friendly
Ship origin: Antarean		 Required action: Friendly
Ship origin: Federation		 Required action: Friendly
Ship origin: Federation		 Required action: Friendly
